In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import random

import matplotlib.pyplot as plt

In [37]:
%pwd

'C:\\Users\\hsoh'

In [38]:
random_seed = 777
torch.manual_seed(random_seed)
random.seed(random_seed)

In [39]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기를 VGGNet의 입력 크기로 변경
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [40]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [41]:
vgg16 = models.vgg16(pretrained=False)
vgg16.classifier[6] = nn.Linear(4096, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [42]:
# 각 레이어의 정보 출력
for name, layer in vgg16.named_children():
    print(f"Layer name: {name}")
    print(layer)

Layer name: features
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2

In [33]:
model_state_dict = vgg16.state_dict()

In [34]:
model_state_dict["features.0.weight"]

tensor([[[[ 0.0016,  0.0220,  0.0005],
          [ 0.0326,  0.0012,  0.0902],
          [ 0.0287, -0.0727,  0.1895]],

         [[-0.0242, -0.0139,  0.0209],
          [ 0.0268, -0.0936,  0.0575],
          [ 0.0542, -0.0360, -0.0199]],

         [[-0.0548, -0.0551, -0.1085],
          [ 0.0754,  0.0285,  0.0153],
          [-0.0530,  0.0223,  0.0212]]],


        [[[ 0.0376, -0.0330,  0.0187],
          [ 0.0545,  0.0303, -0.0809],
          [-0.0387, -0.0702, -0.0724]],

         [[ 0.0432,  0.0232,  0.0671],
          [-0.0730,  0.0059,  0.0175],
          [ 0.1011,  0.1074, -0.0899]],

         [[ 0.0855, -0.0187, -0.1210],
          [ 0.0138, -0.0604,  0.0356],
          [-0.0783, -0.0387,  0.0227]]],


        [[[ 0.1226, -0.0410, -0.1210],
          [-0.0083,  0.0448,  0.0601],
          [ 0.0791,  0.1136,  0.0103]],

         [[-0.0321, -0.0289,  0.0157],
          [ 0.0080,  0.0748, -0.0671],
          [-0.0590, -0.0145,  0.0199]],

         [[ 0.0575,  0.0110,  0.0022],
     

In [43]:
checkpoint_path = f"model_2_inital.pth"
torch.save(vgg16.state_dict(), checkpoint_path)

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.0001)

In [45]:
losses = []
acc = []
for epoch in range(10):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if (i+1) % 50 == 0:    # 매 20 미니배치마다 출력
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0
        
    if epoch + 1 == 5:  # 5 에포크마다 모델 저장
        checkpoint_path = f"model_2_epoch_{epoch + 1}.pth"
        torch.save(vgg16.state_dict(), checkpoint_path)
        print(f"Model saved at epoch {epoch + 1}")
            

    accuracy = 100 * correct / total
    
    losses.append(running_loss / len(trainloader))
    acc.append(accuracy)
    
    print(f"Epoch {epoch + 1}: Loss = {running_loss / len(trainloader):.3f}, Accuracy = {accuracy:.2f}%")

print("Finished Training")

[1,    50] loss: 1.127
[1,   100] loss: 1.024
[1,   150] loss: 0.951
[1,   200] loss: 0.892
[1,   250] loss: 0.853
[1,   300] loss: 0.810
[1,   350] loss: 0.792
[1,   400] loss: 0.750
[1,   450] loss: 0.720
[1,   500] loss: 0.714
[1,   550] loss: 0.693
[1,   600] loss: 0.676
[1,   650] loss: 0.627
[1,   700] loss: 0.622
[1,   750] loss: 0.615
Epoch 1: Loss = 0.048, Accuracy = 42.64%
[2,    50] loss: 0.586
[2,   100] loss: 0.532
[2,   150] loss: 0.556
[2,   200] loss: 0.529
[2,   250] loss: 0.535
[2,   300] loss: 0.500
[2,   350] loss: 0.486
[2,   400] loss: 0.481
[2,   450] loss: 0.472
[2,   500] loss: 0.446
[2,   550] loss: 0.438
[2,   600] loss: 0.452
[2,   650] loss: 0.435
[2,   700] loss: 0.419
[2,   750] loss: 0.410
Epoch 2: Loss = 0.032, Accuracy = 66.14%
[3,    50] loss: 0.355
[3,   100] loss: 0.336
[3,   150] loss: 0.347
[3,   200] loss: 0.328
[3,   250] loss: 0.330
[3,   300] loss: 0.323
[3,   350] loss: 0.322
[3,   400] loss: 0.339
[3,   450] loss: 0.325
[3,   500] loss: 0.32

In [46]:
checkpoint_path = f"model_2_epoch_10.pth"
torch.save(vgg16.state_dict(), checkpoint_path)